Notebook to calculate and save place field correlation mean and hierarchically bootstrapped values

In [1]:
import scipy.stats as sstats
import numpy as np
import matplotlib.pyplot as plt
from os import path
import seaborn as sns
import pandas as pd
import pingouin as pg
from tqdm import tqdm

# Import project specific modules and enable automatic reloading
import sys
from pathlib import Path
import os
eraser_path = Path(os.getcwd()).parent
reinstatement_path = eraser_path.parent / 'FearReinstatement'
sys.path.append(str(eraser_path))
sys.path.append(str(reinstatement_path))
%load_ext autoreload
%autoreload 2

import eraser_reference as err
import er_plot_functions as er
from plotting import Fig, pretty_plot, FigMirror, fix_xlabels
import placefield_stability as pfs
import Placefields as pf
import discrimination as discr
import ca_traces as trc
import cell_tracking as ct
import er_plot_functions as erp
from helpers import flatten
from stats_utils import resample, get_bootstrap_prob, get_bootstrap_prob_paired
from subjects import save_df, load_df

### Definitions

In [2]:
day_dict = {"Before" : [-2, -1], "Before v After" : [-1, 1], "After" : [1, 2], "After2": [2, 7],
            "Before v STM" : [-1, 4], "STM v After" : [4, 1]}

In [3]:
gen_sesh_id = lambda row : f"{row['arena1']}{row['day1']}_{row['arena2']}{row['day2']}"

## 2D Place field correlations

### Before (Day -2 to -1) correlations

In [ ]:
df_bs, corr_df_all = [], []
epoch = 'Before'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        df_list = []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            df_list.append(pfs.pf_corrs_to_df(mouse, arena, day1, arena, day2))
        
        corr_df = pd.concat(df_list)
        corr_df.insert(0, column='Group', value=group_name)
        corr_df.insert(1, column="session_pair", value=corr_df.apply(gen_sesh_id, axis=1))
        corr_df_all.append(corr_df)
        means = []
        for ii in range(10000):
            t = resample(corr_df, level=['mouse', 'session_pair', 'corrs_sm'])
            means.append(t.corrs_sm.mean())
        df_bs.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, "Mean PF Corr": means}))
df_bs = pd.concat(df_bs)
corr_df_all = pd.concat(corr_df_all)
corr_df_all = corr_df_all.rename(columns={"arena1" : "Arena"})
corr_df_all.loc[corr_df_all.Arena == "Open", "Arena"] = "Neutral"

save_df(corr_df_all, "group_pf_corrs_before")
save_df(df_bs, "group_pf_corrs_before_bootstrap")

### Calculate Before v After (Day -1 to Day 1) correlations

In [ ]:
df_bs, corr_df_all = [], []
epoch = 'Before v After'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        df_list = []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            df_list.append(pfs.pf_corrs_to_df(mouse, arena, day1, arena, day2))
        
        corr_df = pd.concat(df_list)
        corr_df.insert(0, column='Group', value=group_name)
        corr_df.insert(1, column="session_pair", value=corr_df.apply(gen_sesh_id, axis=1))
        corr_df_all.append(corr_df)
        means = []
        for ii in range(10000):
            t = resample(corr_df, level=['mouse', 'session_pair', 'corrs_sm'])
            means.append(t.corrs_sm.mean())
        df_bs.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, "Mean PF Corr": means}))
df_bs = pd.concat(df_bs)
corr_df_all = pd.concat(corr_df_all)
corr_df_all = corr_df_all.rename(columns={"arena1" : "Arena"})
corr_df_all.loc[corr_df_all.Arena == "Open", "Arena"] = "Neutral"

save_df(corr_df_all, "group_pf_corrs_before_after")
save_df(df_bs, "group_pf_corrs_before_after_bootstrap")

### After correlations

In [ ]:
df_bs, corr_df_all = [], []
epoch = 'After'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        df_list = []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            df_list.append(pfs.pf_corrs_to_df(mouse, arena, day1, arena, day2))
        
        corr_df = pd.concat(df_list)
        corr_df.insert(0, column='Group', value=group_name)
        corr_df.insert(1, column="session_pair", value=corr_df.apply(gen_sesh_id, axis=1))
        corr_df_all.append(corr_df)
        means = []
        for ii in range(10000):
            t = resample(corr_df, level=['mouse', 'session_pair', 'corrs_sm'])
            means.append(t.corrs_sm.mean())
        df_bs.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, "Mean PF Corr": means}))
df_bs = pd.concat(df_bs)
corr_df_all = pd.concat(corr_df_all)
corr_df_all = corr_df_all.rename(columns={"arena1" : "Arena"})
corr_df_all.loc[corr_df_all.Arena == "Open", "Arena"] = "Neutral"

save_df(corr_df_all, "group_pf_corrs_after")
save_df(df_bs, "group_pf_corrs_after_bootstrap")

In [ ]:
df_bs, corr_df_all = [], []
epoch = 'After2'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        df_list = []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            df_list.append(pfs.pf_corrs_to_df(mouse, arena, day1, arena, day2))
        
        corr_df = pd.concat(df_list)
        corr_df.insert(0, column='Group', value=group_name)
        corr_df.insert(1, column="session_pair", value=corr_df.apply(gen_sesh_id, axis=1))
        corr_df_all.append(corr_df)
        means = []
        for ii in range(10000):
            t = resample(corr_df, level=['mouse', 'session_pair', 'corrs_sm'])
            means.append(t.corrs_sm.mean())
        df_bs.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, "Mean PF Corr": means}))
df_bs = pd.concat(df_bs)
corr_df_all = pd.concat(corr_df_all)
corr_df_all = corr_df_all.rename(columns={"arena1" : "Arena"})
corr_df_all.loc[corr_df_all.Arena == "Open", "Arena"] = "Neutral"

save_df(corr_df_all, "group_pf_corrs_after2")
save_df(df_bs, "group_pf_corrs_after2_bootstrap")

#### STM (4hr) comparisons

In [ ]:
df_bs, corr_df_all = [], []
epoch = 'Before v STM'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        df_list = []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            df_list.append(pfs.pf_corrs_to_df(mouse, arena, day1, arena, day2))
        
        corr_df = pd.concat(df_list)
        corr_df.insert(0, column='Group', value=group_name)
        corr_df.insert(1, column="session_pair", value=corr_df.apply(gen_sesh_id, axis=1))
        corr_df_all.append(corr_df)
        means = []
        for ii in range(10000):
            t = resample(corr_df, level=['mouse', 'session_pair', 'corrs_sm'])
            means.append(t.corrs_sm.mean())
        df_bs.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, "Mean PF Corr": means}))
df_bs = pd.concat(df_bs)
corr_df_all = pd.concat(corr_df_all)
corr_df_all = corr_df_all.rename(columns={"arena1" : "Arena"})
corr_df_all.loc[corr_df_all.Arena == "Open", "Arena"] = "Neutral"

save_df(corr_df_all, "group_pf_corrs_before_stm")
save_df(df_bs, "group_pf_corrs_before_stm_bootstrap")

In [ ]:
df_bs, corr_df_all = [], []
epoch = 'STM v After'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        df_list = []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            df_list.append(pfs.pf_corrs_to_df(mouse, arena, day1, arena, day2))
        
        corr_df = pd.concat(df_list)
        corr_df.insert(0, column='Group', value=group_name)
        corr_df.insert(1, column="session_pair", value=corr_df.apply(gen_sesh_id, axis=1))
        corr_df_all.append(corr_df)
        means = []
        for ii in range(10000):
            t = resample(corr_df, level=['mouse', 'session_pair', 'corrs_sm'])
            means.append(t.corrs_sm.mean())
        df_bs.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, "Mean PF Corr": means}))
df_bs = pd.concat(df_bs)
corr_df_all = pd.concat(corr_df_all)
corr_df_all = corr_df_all.rename(columns={"arena1" : "Arena"})
corr_df_all.loc[corr_df_all.Arena == "Open", "Arena"] = "Neutral"

save_df(corr_df_all, "group_pf_corrs_stm_after")
save_df(df_bs, "group_pf_corrs_stm_after_bootstrap")

### Quick and dirty sanity check plots
Looks good - matches mean data at mouse level from original submission.
real stuff in `Figure2_and_S4-5.ipynb`

In [ ]:
for varname, epoch_name in zip(["before", "before_after", "after", "after2"],
                               ["Before", "Before v After", "After", "After2"]):

    corr_df = load_df(f"group_pf_corrs_{varname}")
    boot_df = load_df(f"group_pf_corrs_{varname}_bootstrap")

    corr_df.loc[corr_df.Arena == "Open", "Arena"] = "Neutral" 
    corr_df["Group"] = pd.Categorical(corr_df["Group"], ["Learners", "Non-Learners", "ANI"])
    corr_df["Arena"] = pd.Categorical(corr_df["Arena"], ["Neutral", "Shock"])
    
    boot_df["Group"] = pd.Categorical(boot_df["Group"], ["Learners", "Non-Learners", "ANI"])
    boot_df["Arena"] = pd.Categorical(boot_df["Arena"], ["Neutral", "Shock"])
    
    
    # corr_means = corr_df.groupby(["Group", "Arena", "mouse"]).mean(numeric_only=True).reset_index()
    # corr_means = corr_df.groupby(["Group", "Arena", "mouse"]).apply(np.nanmean).
    corr_means = corr_df.drop(columns=["session_pair", "day1", "arena2", "day2", "pair_no"]).groupby(["Group", "Arena", "mouse"]).apply(np.nanmean).reset_index()
    corr_means = corr_means.rename(columns={0: "corrs_sm"})
    corr_means["Group"] = pd.Categorical(corr_means["Group"], ["Learners", "Non-Learners", "ANI"])
    corr_means["arena1"] = pd.Categorical(corr_means["Arena"], ["Neutral", "Shock"])
    
    _, ax = plt.subplots()
    pal_use = sns.color_palette(palette='Set2', as_cmap=False)[0:3] 
    sns.boxplot(data=boot_df, x="Arena", y="Mean PF Corr", hue="Group", showfliers=False, fill=False, palette=pal_use, ax=ax)
    sns.stripplot(data=corr_means, x="Arena", y="corrs_sm", hue="Group", dodge=True, palette=pal_use, linewidth=0.2, edgecolor='w',
                  ax=ax, legend=False)
    sns.despine(ax=ax)
    ax.set_title(epoch_name)

# PV1d correlation bootstraps

In [ ]:
rng = np.random.default_rng()
resample_mice = rng.choice(mice, size=len(mice), replace=True)
resample_mice


In [5]:
def resample_pv1d(mice, arena, day1, day2, shuf_map=False):
    rng = np.random.default_rng()
    resample_mice = rng.choice(mice, size=len(mice), replace=True)

    pv_corrs_both, pv_corrs_all = [], []
    for mouse in mice:
        # day1, day2 = day_dict[epoch]
        corr_all, corr_both = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=True, shuf_map=shuf_map)
        pv_corrs_both.append(corr_both)
        pv_corrs_all.append(corr_all)

    return np.mean(pv_corrs_both), np.mean(pv_corrs_all)
resample_pv1d(err.learners, 'Shock', 1, 2)

(0.3338316671803213, -0.20040189480599838)

In [ ]:
# Code to test parallelization

from joblib import Parallel, delayed
import functools

arena = 'Shock'
day1, day2 = 1, 2

partial_resample = functools.partial(resample_pv1d, arena=arena, day1=day1, day2=day2)
n_iter = 10000
n_jobs = 8
out_df = []

data = [
    r
    for r in tqdm(
        Parallel(n_jobs=n_jobs, return_as="generator")(
            delayed(partial_resample)(err.learners) for _ in range(n_iter)
        ),
        total=n_iter,
        # position=1,
        # leave=False,
    )
]

In [6]:
from joblib import Parallel, delayed
import functools
def bootstrap_resample_pv1d(mice, arena, day1, day2, group, comparison, arena_rename, shuf_map=False, n_iter=1000, n_jobs=12):
    partial_resample = functools.partial(resample_pv1d, arena=arena, day1=day1, day2=day2, shuf_map=shuf_map)

    data = [
        r
        for r in tqdm(
            Parallel(n_jobs=n_jobs, return_as="generator")(
                delayed(partial_resample)(mice) for _ in range(n_iter)
            ),
            total=n_iter,
            # position=1,
            # leave=False,
        )
    ]
    return pd.DataFrame({"Group": group, "Arena": arena_rename, "Comparison": comparison, 
                         "Mean 1d PV Corr (Both)": np.array(data)[:, 0], 	"Mean 1d PV Corr (All)": np.array(data)[:,1]})

In [ ]:

for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        pv_means_both, pv_means_all = [], []
        print(f'resampling for {arena_name} {group_name}')

        day1, day2 = day_dict["Before"]
        data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, n_iter=n_iter)
        boot_df.append(data)
boot_df = pd.concat(boot_df)

In [ ]:
# %%timeit
n_iter = 10000
df_bs, corr_df_all = [], []
epoch = 'Before'
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        pv_means_both, pv_means_all = [], []
        print(f'resampling for {arena_name} {group_name}')

        day1, day2 = day_dict[epoch]
        data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, n_iter=n_iter)
        df_bs.append(data)
df_bs = pd.concat(df_bs, ignore_index=True)

# Do the same for session means without bootstrapping
pvcorr_df = []
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                            [err.learners, err.nonlearners, err.ani_mice_good]):

        pv_corrs_both, pv_corrs_all = [], []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            corr_all, corr_both = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=False)
            pv_corrs_both.append(corr_both)
            pv_corrs_all.append(corr_all)
        pvcorr_df.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, 
                                   "1d PV Corr (Both)": pv_corrs_both, "1d PV Corr (All)": pv_corrs_all}))

pvcorr_df = pd.concat(pvcorr_df, ignore_index=True)

save_df(pvcorr_df, "group_pv1d_corrs_before")
save_df(df_bs, "group_pv1d_corrs_before_bootstrap")

In [ ]:

df_bs, corr_df_all = [], []
epoch = 'Before v After'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        pv_means_both, pv_means_all = [], []
        print(f'resampling for {arena_name} {group_name}')

        day1, day2 = day_dict[epoch]
        data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, n_iter=n_iter)
        df_bs.append(data)
df_bs = pd.concat(df_bs, ignore_index=True)

# Do the same for session means without bootstrapping
pvcorr_df = []
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                            [err.learners, err.nonlearners, err.ani_mice_good]):

        pv_corrs_both, pv_corrs_all = [], []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            corr_all, corr_both = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=False)
            pv_corrs_both.append(corr_both)
            pv_corrs_all.append(corr_all)
        pvcorr_df.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, 
                                   "1d PV Corr (Both)": pv_corrs_both, "1d PV Corr (All)": pv_corrs_all}))

pvcorr_df = pd.concat(pvcorr_df, ignore_index=True)

save_df(pvcorr_df, "group_pv1d_corrs_before_after")
save_df(df_bs, "group_pv1d_corrs_before_after_bootstrap")

In [ ]:

df_bs, corr_df_all = [], []
epoch = 'After'
resample_levels = ["mouse", "session", "corrs_sm"]
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        pv_means_both, pv_means_all = [], []
        print(f'resampling for {arena_name} {group_name}')

        day1, day2 = day_dict[epoch]
        data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, n_iter=n_iter)
        df_bs.append(data)
df_bs = pd.concat(df_bs, ignore_index=True)

# Do the same for session means without bootstrapping
pvcorr_df = []
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                            [err.learners, err.nonlearners, err.ani_mice_good]):

        pv_corrs_both, pv_corrs_all = [], []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            corr_all, corr_both = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=False)
            pv_corrs_both.append(corr_both)
            pv_corrs_all.append(corr_all)
        pvcorr_df.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, 
                                   "1d PV Corr (Both)": pv_corrs_both, "1d PV Corr (All)": pv_corrs_all}))

pvcorr_df = pd.concat(pvcorr_df, ignore_index=True)

save_df(pvcorr_df, "group_pv1d_corrs_after")
save_df(df_bs, "group_pv1d_corrs_after_bootstrap")

In [ ]:

df_bs, corr_df_all = [], []
epoch = 'Before v STM'
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        pv_means_both, pv_means_all = [], []
        print(f'resampling for {arena_name} {group_name}')

        day1, day2 = day_dict[epoch]
        data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, n_iter=n_iter)
        df_bs.append(data)
df_bs = pd.concat(df_bs, ignore_index=True)

# Do the same for session means without bootstrapping
pvcorr_df = []
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                            [err.learners, err.nonlearners, err.ani_mice_good]):

        pv_corrs_both, pv_corrs_all = [], []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            corr_all, corr_both = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=False)
            pv_corrs_both.append(corr_both)
            pv_corrs_all.append(corr_all)
        pvcorr_df.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, 
                                   "1d PV Corr (Both)": pv_corrs_both, "1d PV Corr (All)": pv_corrs_all}))

pvcorr_df = pd.concat(pvcorr_df, ignore_index=True)

save_df(pvcorr_df, "group_pv1d_corrs_before_stm")
save_df(df_bs, "group_pv1d_corrs_before_stm_bootstrap")

In [ ]:

df_bs, corr_df_all = [], []
epoch = 'STM v After'
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        pv_means_both, pv_means_all = [], []
        print(f'resampling for {arena_name} {group_name}')

        day1, day2 = day_dict[epoch]
        data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, n_iter=n_iter)
        df_bs.append(data)
df_bs = pd.concat(df_bs, ignore_index=True)

# Do the same for session means without bootstrapping
pvcorr_df = []
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                            [err.learners, err.nonlearners, err.ani_mice_good]):

        pv_corrs_both, pv_corrs_all = [], []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            corr_all, corr_both = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=False)
            pv_corrs_both.append(corr_both)
            pv_corrs_all.append(corr_all)
        pvcorr_df.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, 
                                   "1d PV Corr (Both)": pv_corrs_both, "1d PV Corr (All)": pv_corrs_all}))

pvcorr_df = pd.concat(pvcorr_df, ignore_index=True)

save_df(pvcorr_df, "group_pv1d_corrs_stm_after")
save_df(df_bs, "group_pv1d_corrs_stm_after_bootstrap")

In [38]:
n_iter = 1000

In [41]:

df_bs, df_shuf_bs, corr_df_all = [], [], []
epoch = 'After2'
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                                [err.learners, err.nonlearners, err.ani_mice_good]):
        pv_means_both, pv_means_all = [], []
        print(f'resampling for {arena_name} {group_name}')

        day1, day2 = day_dict[epoch]
        data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, n_iter=n_iter)
        shuf_data = bootstrap_resample_pv1d(mice, arena, day1, day2, group_name, epoch, arena_name, shuf_map=True, n_iter=n_iter)
        df_bs.append(data)
        df_shuf_bs.append(shuf_data)
df_bs = pd.concat(df_bs, ignore_index=True)
df_shuf_bs = pd.concat(df_shuf_bs, ignore_index=True)

# Merge shuffled values into main DataFrame
df_bs = df_bs.merge(df_shuf_bs.rename(columns={"Mean 1d PV Corr (Both)" : "Shuf 1d PV Corr (Both)", 
                                       "Mean 1d PV Corr (All)": "Shuf 1d PV Corr (All)"}),
            right_index=True, left_index=True, copy=False, suffixes=(None, "2")).drop(columns=["Group2", "Arena2", "Comparison2"])
# Do the same for session means without bootstrapping
pvcorr_df = []
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                            [err.learners, err.nonlearners, err.ani_mice_good]):

        pv_corrs_both, pv_corrs_all = [], []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            corr_all, corr_both = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=False)
            pv_corrs_both.append(corr_both)
            pv_corrs_all.append(corr_all)
        pvcorr_df.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, 
                                   "1d PV Corr (Both)": pv_corrs_both, "1d PV Corr (All)": pv_corrs_all}))

pvcorr_df = pd.concat(pvcorr_df, ignore_index=True)

save_df(pvcorr_df, "group_pv1d_corrs_after2")
save_df(df_bs, "group_pv1d_corrs_after2_bootstrap")
# save_df(df_shuf_bs, "group_pv1d_shuf_corrs_after2_bootstrap")

resampling for Shock Learners


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 55.98it/s]


resampling for Shock Non-Learners


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:30<00:00, 32.60it/s]


resampling for Shock ANI


...

resampling for Neutral Non-Learners


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:43<00:00, 23.22it/s]


resampling for Neutral ANI


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.42it/s]


group_pv1d_corrs_after2.csv saved
group_pv1d_corrs_after2_bootstrap.csv saved


In [ ]:
pvcorr_df = []
for arena, arena_name in zip(['Shock', 'Open'], ['Shock', 'Neutral']):
    for group_name, mice in zip(["Learners", "Non-Learners", "ANI"], 
                            [err.learners, err.nonlearners, err.ani_mice_good]):

        pv_corrs_both, pv_corrs_all = [], []
        for mouse in mice:
            day1, day2 = day_dict[epoch]
            corr_both, corr_all = pfs.PV1_corr_bw_sesh(mouse, arena, day1, arena, day2, bootstrap=False)
            pv_corrs_both.append(corr_both)
            pv_corrs_all.append(corr_all)
        pvcorr_df.append(pd.DataFrame({"Group": group_name, "Arena": arena_name, "Comparison": epoch, 
                                   "1d PV Corr (Both)": pv_corrs_both, "1d PV Corr (All)": pv_corrs_all}))

In [ ]:
pd.concat(pvcorr_df, ignore_index=True)